In [ ]:
!adk web --log_level DEBUG path/to/your/agents_dir

In [ ]:
!adk web --help


In [ ]:
!adk web --log_level DEBUG

In [1]:
# Check the DEBUG logs from the broken agent
print("🔍 Examining web server logs for debugging clues...\n")
!cat logger.log

🔍 Examining web server logs for debugging clues...

cat: logger.log: No such file or directory
